In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-house-qr)


We will use Householder reflections to produce a QR factorization of a random matrix.
```{tip}
:class: dropdown
The `rand` function can select randomly from within the interval $[0,1]$, or from a vector or range that you specify.
```

In [2]:
A = rand(float(1:9), 6, 4)
m,n = size(A)

(6, 4)

```{index} Julia; normalize, ! Julia; I
```

Our first step is to introduce zeros below the diagonal in column 1 by using {eq}`hhvector` and {eq}`hhreflect`. 
:::
```{tip}
:class: dropdown
`I` can stand for an identity matrix of any size, inferred from the context when needed.
```

In [3]:
z = A[:, 1];
v = normalize(z - norm(z) * [1; zeros(m-1)])
P₁ = I - 2v * v'   # reflector

6×6 Matrix{Float64}:
 0.395628   0.395628   0.593442   0.395628   0.263752   0.32969
 0.395628   0.741017  -0.388474  -0.258983  -0.172655  -0.215819
 0.593442  -0.388474   0.417289  -0.388474  -0.258983  -0.323728
 0.395628  -0.258983  -0.388474   0.741017  -0.172655  -0.215819
 0.263752  -0.172655  -0.258983  -0.172655   0.884897  -0.143879
 0.32969   -0.215819  -0.323728  -0.215819  -0.143879   0.820151

We check that this reflector introduces zeros as it should:

In [4]:
P₁ * z

6-element Vector{Float64}:
 15.165750888103098
  1.4988010832439613e-15
  2.942091015256665e-15
  1.4988010832439613e-15
  6.106226635438361e-16
  1.3322676295501878e-15

Now we replace $\mathbf{A}$ by $\mathbf{P}\mathbf{A}$.

In [5]:
A = P₁ * A

6×4 Matrix{Float64}:
 15.1658       13.5832    11.4073   13.0557
  1.4988e-15   -0.237124  -2.15811   5.69046
  2.94209e-15  -3.35569   -1.23716  -2.96431
  1.4988e-15   -1.23712   -2.15811  -1.30954
  6.10623e-16  -4.49142    4.89459   6.79364
  1.33227e-15  -1.86427   -4.13176   3.24205

We are set to put zeros into column 2. We must not use row 1 in any way, lest it destroy the zeros we just introduced. So we leave it out of the next reflector.

In [6]:
z = A[2:m, 2]
v = normalize(z - norm(z) * [1; zeros(m-2)])
P₂ = I - 2v * v'

5×5 Matrix{Float64}:
 -0.0392514  -0.55547   -0.204782   -0.743469  -0.308595
 -0.55547     0.703106  -0.109454   -0.397377  -0.164941
 -0.204782   -0.109454   0.959648   -0.146499  -0.0608079
 -0.743469   -0.397377  -0.146499    0.468131  -0.220765
 -0.308595   -0.164941  -0.0608079  -0.220765   0.908366

We now apply this reflector to rows 2 and below only.

In [7]:
A[2:m, :] = P₂ * A[2:m, :]
A

6×4 Matrix{Float64}:
 15.1658       13.5832       11.4073    13.0557
 -2.86511e-15   6.04116      -1.15008   -4.35994
  6.0962e-16    7.83721e-16  -0.698382  -8.33616
  6.38901e-16   2.45104e-16  -1.95948   -3.28995
 -2.51127e-15   1.07526e-15   5.61573   -0.396306
  3.64507e-17   4.14496e-16  -3.83243    0.257692

We need to iterate the process for the last two columns.

In [8]:
for j in 3:n
    z = A[j:m, j]
    v = normalize(z - norm(z) * [1; zeros(m-j)])
    P = I - 2v * v'
    A[j:m, :] = P * A[j:m, :]
end

We have now reduced the original to an upper triangular matrix using four orthogonal Householder reflections:

In [9]:
R = triu(A)

6×4 Matrix{Float64}:
 15.1658  13.5832   11.4073   13.0557
  0.0      6.04116  -1.15008  -4.35994
  0.0      0.0       7.10994   1.27361
  0.0      0.0       0.0       8.8835
  0.0      0.0       0.0       0.0
  0.0      0.0       0.0       0.0